In [1]:
import json
from tqdm import tqdm
import random

In [2]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/sft_data/llava187k.json", 'r', encoding='utf-8') as file:
    dpo_data = json.load(file)
print(len(dpo_data))

187383


In [3]:
dpo_data[1000]

{'id': '000000178859',
 'image': 'coco/train2017/000000178859.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat type of animals are in the image?'},
  {'from': 'gpt', 'value': 'There are horses in the image.'},
  {'from': 'human',
   'value': 'What is the terrain like where the horses are standing?'},
  {'from': 'gpt',
   'value': 'The horses are standing in a grassy area beside a small rock wall.'},
  {'from': 'human',
   'value': 'Are the horses near any body of water or mountains?'},
  {'from': 'gpt',
   'value': 'Yes, the horses are in a rock-walled pasture by a lake in the mountains.'},
  {'from': 'human', 'value': 'How many horses are visible in the image?'},
  {'from': 'gpt',
   'value': 'There are a couple of horses visible in the image.'},
  {'from': 'human',
   'value': 'What could be the purpose of the rock wall near the horses?'},
  {'from': 'gpt',
   'value': "The rock wall near the horses may serve a few purposes. One possible purpose is to act as a bo

In [4]:
"""
textvqa 数据集全部都是以 <image>\nProvide a one-sentence caption for the provided image.\n 作为问题的开头
"""
count = 0
count_convs = 0
textvqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'textvqa':
        textvqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
        if len(data["conversations"])==2:
            count_convs += 1
print(count)
print(count_convs)
print(len(textvqa_data))

2195
2195
2195


In [5]:
"""
coco 数据集全部都是以 <image>\n 或者是 <image> 作为问题的开头或者是结尾
"""
count = 0
coco_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'coco':
        coco_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
        if data["conversations"][0]['value'].endswith('<image>'):
            count+=1
print(count)
print(len(coco_data))

36410
36410


In [6]:
"""
ocr_vqa 全部都是以 <image>\n 作为问题的开头
"""
count = 0
ocr_vqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'ocr_vqa':
        ocr_vqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
print(count)
print(len(ocr_vqa_data))

8000
8000


In [7]:
"""
vg 全部都是以 <image>\n 作为问题的开头
"""
count = 0
vg_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'vg':
        vg_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
print(count)
print(len(vg_data))

8642
8642


In [8]:
"""
gpa 全部都是以 <image>\n 作为问题的开头
"""
count = 0
statistic = 0
gqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'gqa':
        gqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n') and data["conversations"][0]['value'].endswith('Answer the question using a single word or phrase.'):
            count+=1
print(count)
print(len(gqa_data))

7214
7214


In [ ]:
"""
构造 textvqa + coco 45k的数据集
"""

In [6]:
def find_companions(input_str, str_list, count=1):
    filtered_list = [s for s in str_list if s != input_str]
    return random.sample(filtered_list, count)

In [7]:
textvqa_image = []
for data in dpo_data[:20000]:
    textvqa_image.append(data["image"])
print(len(textvqa_image))

coco_image = []
for data in dpo_data[20000:]:
    coco_image.append(data["image"])
print(len(coco_image))

20000
25000


In [8]:
mix_data = []
for index, data in enumerate(dpo_data[:20000]):
    if index<11000:
        image_list = find_companions(data['image'], textvqa_image, 1)
    elif index<14000:
        image_list = find_companions(data['image'], textvqa_image, 2)
    elif index<17000:
        image_list = find_companions(data['image'], textvqa_image, 3)
    elif index<20000:
        image_list = find_companions(data['image'], textvqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    if image_length == 2:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 3:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 4:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 5:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    mix_data.append(data)
    
file_path = "./mix_testvqa_20k.json"
with open(file_path, 'w') as file:
    json.dump(mix_data, file, indent=4)
print(f'数据已保存到 {file_path}')

数据已保存到 ./mix_testvqa_20k.json


In [9]:
mix_data = []
for index, data in enumerate(dpo_data[20000:]):
    if index<16000:
        image_list = find_companions(data['image'], coco_image, 1)
    elif index<19000:
        image_list = find_companions(data['image'], coco_image, 2)
    elif index<22000:
        image_list = find_companions(data['image'], coco_image, 3)
    elif index<25000:
        image_list = find_companions(data['image'], coco_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    mix_data.append(data)
    
file_path = "./mix_coco_25k.json"
with open(file_path, 'w') as file:
    json.dump(mix_data, file, indent=4)
print(f'数据已保存到 {file_path}')

数据已保存到 ./mix_coco_25k.json


In [10]:
import json

def merge_json_files(file_path1, file_path2, output_file_path):
    # 读取第一个JSON文件
    with open(file_path1, 'r') as file:
        data1 = json.load(file)
    
    # 读取第二个JSON文件
    with open(file_path2, 'r') as file:
        data2 = json.load(file)
    
    # 合并两个字典
    merged_list = data1 + data2  # 列表拼接
    
    # 将合并后的数据写入新的JSON文件
    with open(output_file_path, 'w') as file:
        json.dump(merged_list, file, indent=4)


In [11]:
# JSON文件路径
file_path1 = './mix_testvqa_20k.json'
file_path2 = './mix_coco_25k.json'
output_file_path = './mix_textvqa_20k_coco_25k.json'

# 调用函数合并文件
merge_json_files(file_path1, file_path2, output_file_path)

In [12]:
# JSON文件路径
file_path1 = './mix_textvqa_20k_coco_25k.json'
file_path2 = '/mnt/hwfile/mllm/chenlin/llava/data/llava/llava_v1_5_mix665k.json'
output_file_path = './llava_all665k_textvqa_20k_coco_25k.json'

# 调用函数合并文件
merge_json_files(file_path1, file_path2, output_file_path)

In [ ]:
"""
变动 textvqa 和 coco 数据集的数据组合比例
"""

In [4]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_textvqa_20k_coco_25k.json", 'r', encoding='utf-8') as file:
    dpo_data = json.load(file)
print(len(dpo_data))

45000


In [5]:
mix_textvqa_20k_coco_20k = dpo_data[:40000]
print(len(mix_textvqa_20k_coco_20k))

40000


In [6]:
file_path = "./mix_textvqa_20k_coco_20k.json"
with open(file_path, 'w') as file:
    json.dump(mix_textvqa_20k_coco_20k, file, indent=4)
print("Done")

Done


In [ ]:
"""
变动数据比例
"""

In [8]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_textvqa_20k_coco_25k.json", 'r', encoding='utf-8') as file:
    dpo_data = json.load(file)
print(len(dpo_data))

45000


In [9]:
mix_textvqa_15k_coco_25k = dpo_data[5000:]
print(len(mix_textvqa_15k_coco_25k))

40000


In [10]:
file_path = "./mix_textvqa_15k_coco_25k.json"
with open(file_path, 'w') as file:
    json.dump(mix_textvqa_15k_coco_25k, file, indent=4)
print("Done")

Done


In [ ]:
"""
变动数据比例
"""

In [11]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_textvqa_20k_coco_25k.json", 'r', encoding='utf-8') as file:
    dpo_data = json.load(file)
print(len(dpo_data))

45000


In [12]:
mix_textvqa_17k_coco_22k = dpo_data[3000:42000]
print(len(mix_textvqa_17k_coco_22k))

39000


In [13]:
file_path = "./mix_textvqa_17k_coco_22k.json"
with open(file_path, 'w') as file:
    json.dump(mix_textvqa_17k_coco_22k, file, indent=4)
print("Done")

Done


In [ ]:
"""
从 llava665k 中选取了 62k 的数据，并组合为多图数据
"""

In [9]:
def find_companions(input_str, str_list, count=1):
    filtered_list = [s for s in str_list if s != input_str]
    return random.sample(filtered_list, count)

In [10]:
textvqa_image = []
for data in textvqa_data:
    textvqa_image.append(data["image"])
print(len(textvqa_image))

coco_image = []
for data in coco_data:
    coco_image.append(data["image"])
print(len(coco_image))

ocr_vqa_image = []
for data in ocr_vqa_data:
    ocr_vqa_image.append(data["image"])
print(len(ocr_vqa_image))

vq_image = []
for data in vg_data:
    vq_image.append(data["image"])
print(len(vq_image))

gqa_image = []
for data in gqa_data:
    gqa_image.append(data["image"])
print(len(gqa_image))

all_images = textvqa_image + coco_image + ocr_vqa_image + vq_image + gqa_image
print(len(all_images))

2195
36410
8000
8642
7214
62461


In [13]:
mix_textvqa_data = []
for index, data in enumerate(textvqa_data):
    if index<1200:
        image_list = find_companions(data['image'], textvqa_image, 1)
    elif index<1550:
        image_list = find_companions(data['image'], textvqa_image, 2)
    elif index<1900:
        image_list = find_companions(data['image'], textvqa_image, 3)
    elif index<2195:
        image_list = find_companions(data['image'], textvqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    if image_length == 2:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 3:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 4:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 5:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    mix_textvqa_data.append(data)

print(len(mix_textvqa_data))

2195


In [14]:
mix_coco_data = []
for index, data in enumerate(coco_data):
    if index<20000:
        image_list = find_companions(data['image'], coco_image, 1)
    elif index<26000:
        image_list = find_companions(data['image'], coco_image, 2)
    elif index<32000:
        image_list = find_companions(data['image'], coco_image, 3)
    elif index<36410:
        image_list = find_companions(data['image'], coco_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    mix_coco_data.append(data)
print(len(mix_coco_data))

36410


In [15]:
mix_ocr_vqa_data = []
for index, data in enumerate(ocr_vqa_data):
    if index<4500:
        image_list = find_companions(data['image'], ocr_vqa_image, 1)
    elif index<5700:
        image_list = find_companions(data['image'], ocr_vqa_image, 2)
    elif index<6900:
        image_list = find_companions(data['image'], ocr_vqa_image, 3)
    elif index<8000:
        image_list = find_companions(data['image'], ocr_vqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    mix_ocr_vqa_data.append(data)
print(len(mix_ocr_vqa_data))

8000


In [16]:
mix_vg_data = []
for index, data in enumerate(vg_data):
    if index<4800:
        image_list = find_companions(data['image'], vq_image, 1)
    elif index<6100:
        image_list = find_companions(data['image'], vq_image, 2)
    elif index<7400:
        image_list = find_companions(data['image'], vq_image, 3)
    elif index<8642:
        image_list = find_companions(data['image'], vq_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    mix_vg_data.append(data)
print(len(mix_vg_data))

8642


In [17]:
mix_gqa_data = []
for index, data in enumerate(gqa_data):
    if index<3800:
        image_list = find_companions(data['image'], gqa_image, 1)
    elif index<5000:
        image_list = find_companions(data['image'], gqa_image, 2)
    elif index<6100:
        image_list = find_companions(data['image'], gqa_image, 3)
    elif index<7214:
        image_list = find_companions(data['image'], gqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    mix_gqa_data.append(data)
print(len(mix_gqa_data))

7214


In [18]:
mix_data = mix_textvqa_data + mix_coco_data + mix_ocr_vqa_data + mix_vg_data + mix_gqa_data
print(len(mix_data))
file_path = "./mix_llava62k_dpo_v2.json"
with open(file_path, 'w') as file:
    json.dump(mix_data, file, indent=4)
print("Done")

62461
Done


In [ ]:
"""
从 llava665k 中选取了 62k 的数据，并组合拼图数据
"""

In [9]:
def find_companions(input_str, str_list, count=1):
    filtered_list = [s for s in str_list if s != input_str]
    return random.sample(filtered_list, count)

In [10]:
from PIL import Image, ImageDraw, ImageFont

def resize_and_pad(image, target_size):
    """Resize and pad an image to the target size."""
    img_ratio = image.width / image.height
    target_ratio = target_size[0] / target_size[1]

    if img_ratio > target_ratio:
        new_width = target_size[0]
        new_height = int(new_width / img_ratio)
    else:
        new_height = target_size[1]
        new_width = int(new_height * img_ratio)

    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    new_image = Image.new("RGB", target_size, (255, 255, 255))
    # paste_x = (target_size[0] - new_width) // 2
    # paste_y = (target_size[1] - new_height) // 2
    paste_x = 0
    paste_y = 0
    new_image.paste(resized_image, (paste_x, paste_y))
    return new_image

def combine_images_with_labels(image_paths, output_path, target_size=(448, 448), margin=40):
    # # 获取提问的图像的大小
    # resize_size = Image.open(image_paths[image_index-1]).size
    # print(resize_size)
    # # Load images and resize them
    # images = [Image.open(image_path).resize(resize_size) for image_path in image_paths]
    # num_images = len(images)

    images = [resize_and_pad(Image.open(image_path), target_size) for image_path in image_paths]
    num_images = len(images)

    resize_size = target_size
    # Calculate combined image size
    if num_images == 2 or num_images == 3:
        combined_width = resize_size[0] * num_images + margin * (num_images - 1)
        combined_height = resize_size[1] + margin  # Add margin for labels
    elif num_images == 4:
        combined_width = resize_size[0] * 2 + margin
        combined_height = resize_size[1] * 2 + margin * 2  # Add margin for labels
    elif num_images == 6:
        combined_width = resize_size[0] * 3 + margin * 2
        combined_height = resize_size[1] * 2 + margin * 2  # Add margin for labels
    elif num_images == 9:
        combined_width = resize_size[0] * 3 + margin * 2
        combined_height = resize_size[1] * 3 + margin * 3  # Add margin for labels
    else:
        raise ValueError("The number of images must be 2, 3, 4, 6, or 9.")
    
    # Create new image with white background
    combined_image = Image.new('RGB', (combined_width, combined_height), (255, 255, 255))
    
    # Define positions for each image and label
    positions = []
    labels = []
    if num_images == 2 or num_images == 3:
        for i in range(num_images):
            x = i * (resize_size[0] + margin)
            y = 0+margin
            positions.append((x, y))
            labels.append(f"Image{i+1}")
    elif num_images == 4:
        for i in range(2):
            for j in range(2):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{2*i+j+1}")
    elif num_images == 6:
        for i in range(2):
            for j in range(3):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{3*i+j+1}")
    elif num_images == 9:
        for i in range(3):
            for j in range(3):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{3*i+j+1}")
    
    # Paste images and draw labels
    draw = ImageDraw.Draw(combined_image)
    font = ImageFont.truetype("Arial.ttf", 36)  # Load a larger TrueType font, or use another available font

    for img, pos, label in zip(images, positions, labels):
        combined_image.paste(img, pos)
        # Calculate label position in the left top corner
        label_x = pos[0] + 20  # Add some padding from the left edge
        label_y = pos[1] - 40  # Add some padding from the top edge
        draw.text((label_x, label_y), label, (0, 0, 0), font=font)
    
    # Save the combined image
    combined_image.save(output_path)


# # Example usage
# image_list = [
#     "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/car1.jpg",
#     "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/car2.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/newcar3.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/newcar4.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/bear.png",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
# ]
# output_path = "./combined_image_9.png"
# combine_images_with_labels(image_list, output_path)
# print("Done")

In [11]:
textvqa_image = []
for data in textvqa_data:
    textvqa_image.append(data["image"])
print(len(textvqa_image))

coco_image = []
for data in coco_data:
    coco_image.append(data["image"])
print(len(coco_image))

ocr_vqa_image = []
for data in ocr_vqa_data:
    ocr_vqa_image.append(data["image"])
print(len(ocr_vqa_image))

vq_image = []
for data in vg_data:
    vq_image.append(data["image"])
print(len(vq_image))

gqa_image = []
for data in gqa_data:
    gqa_image.append(data["image"])
print(len(gqa_image))

all_images = textvqa_image + coco_image + ocr_vqa_image + vq_image + gqa_image
print(len(all_images))

2195
36410
8000
8642
7214
62461


In [12]:
mix_textvqa_data = []
for index, data in tqdm(enumerate(textvqa_data)):
    if index<1000:
        image_list = find_companions(data['image'], textvqa_image, 1)
    elif index<1300:
        image_list = find_companions(data['image'], textvqa_image, 2)
    elif index<1600:
        image_list = find_companions(data['image'], textvqa_image, 3)
    elif index<1900:
        image_list = find_companions(data['image'], textvqa_image, 5)
    elif index<2195:
        image_list = find_companions(data['image'], textvqa_image, 8)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_v2/textvqa_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', '<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    data['image'] = output_path
    mix_textvqa_data.append(data)

print(len(mix_textvqa_data))
output_json_path = "./mix_textvqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_textvqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

2195it [02:59, 12.24it/s]


2195
Data has been written to ./mix_textvqa_data.json


In [13]:
mix_ocr_vqa_data = []
for index, data in tqdm(enumerate(ocr_vqa_data)):
    if index<3800:
        image_list = find_companions(data['image'], ocr_vqa_image, 1)
    elif index<4900:
        image_list = find_companions(data['image'], ocr_vqa_image, 2)
    elif index<6000:
        image_list = find_companions(data['image'], ocr_vqa_image, 3)
    elif index<7100:
        image_list = find_companions(data['image'], ocr_vqa_image, 5)
    elif index<8000:
        image_list = find_companions(data['image'], ocr_vqa_image, 8)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_v2/ocrvqa_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                convs['value'] = '<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data['image'] = output_path
    mix_ocr_vqa_data.append(data)
    
print(len(mix_ocr_vqa_data))
output_json_path = "./mix_ocr_vqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_ocr_vqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

8000it [05:18, 25.10it/s]


8000
Data has been written to ./mix_ocr_vqa_data.json


In [14]:
mix_gqa_data = []
for index, data in tqdm(enumerate(gqa_data)):
    if index<3200:
        image_list = find_companions(data['image'], gqa_image, 1)
    elif index<4300:
        image_list = find_companions(data['image'], gqa_image, 2)
    elif index<5400:
        image_list = find_companions(data['image'], gqa_image, 3)
    elif index<6300:
        image_list = find_companions(data['image'], gqa_image, 5)
    elif index<7214:
        image_list = find_companions(data['image'], gqa_image, 8)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_v2/gqa_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index += 1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                convs['value'] = '<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data['image'] = output_path
    mix_gqa_data.append(data)
    
print(len(mix_gqa_data))
output_json_path = "./mix_gqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_gqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

7214it [05:35, 21.48it/s]


7214
Data has been written to ./mix_gqa_data.json


In [15]:
import re
def contains_bbox(s):
    bbox_pattern = r'\[\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?\s*\]'
    match = re.search(bbox_pattern, s)
    if match:
        return True
    else:
        return False

In [16]:
mix_coco_data = []
for index, data in tqdm(enumerate(coco_data)):
    choose_flase = True
    for conv in data["conversations"]:
        conv_value = conv["value"]
        if contains_bbox(conv_value):
            choose_flase = False
    if choose_flase == True: 
        if index<18000:
            image_list = find_companions(data['image'], coco_image, 1)
        elif index<23000:
            image_list = find_companions(data['image'], coco_image, 2)
        elif index<28000:
            image_list = find_companions(data['image'], coco_image, 3)
        elif index<32000:
            image_list = find_companions(data['image'], coco_image, 5)
        elif index<36410:
            image_list = find_companions(data['image'], coco_image, 8)
        image_list.append(data['image'])
        random.shuffle(image_list)
        image_index = image_list.index(data['image'])
        new_image_list = []
        for img_path in image_list:
            new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
        output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_v2/coco_{index}.jpg'
        combine_images_with_labels(new_image_list, output_path)
    
        image_index += 1
        image_length = len(image_list)
        data['image'] = image_list
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
        for convs_index, convs in enumerate(data["conversations"]):
            if convs['from'] == 'human':
                if convs_index == 0:
                    convs['value'] = '<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                else:
                    convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
        data['image'] = output_path
        mix_coco_data.append(data)
        
print(len(mix_coco_data))
output_json_path = "./mix_coco_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_coco_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

36410it [22:36, 26.85it/s]


31565
Data has been written to ./mix_coco_data.json


In [17]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_coco_data.json", 'r', encoding='utf-8') as file:
    mix_coco_data = json.load(file)
print(len(mix_coco_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_gqa_data.json", 'r', encoding='utf-8') as file:
    mix_gqa_data = json.load(file)
print(len(mix_gqa_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_ocr_vqa_data.json", 'r', encoding='utf-8') as file:
    mix_ocr_vqa_data = json.load(file)
print(len(mix_ocr_vqa_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_textvqa_data.json", 'r', encoding='utf-8') as file:
    mix_textvqa_data = json.load(file)
print(len(mix_textvqa_data))
ping_llava62k_v2 = mix_coco_data+mix_gqa_data+mix_ocr_vqa_data+mix_textvqa_data
print(len(ping_llava62k_v2))
output_json_path = "./ping_llava62k_dpo_v3.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_llava62k_v2, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

31565
7214
8000
2195
48974
Data has been written to ./ping_llava62k_dpo_v3.json


In [ ]:
"""
从 llava665k 中选取了 62k 的数据，抽取50分之一，构造拒答数据
"""

In [9]:
def find_companions(input_str, str_list, count=1):
    filtered_list = [s for s in str_list if s != input_str]
    return random.sample(filtered_list, count)

In [10]:
textvqa_image = []
for data in textvqa_data:
    textvqa_image.append(data["image"])
print(len(textvqa_image))

coco_image = []
for data in coco_data:
    coco_image.append(data["image"])
print(len(coco_image))

ocr_vqa_image = []
for data in ocr_vqa_data:
    ocr_vqa_image.append(data["image"])
print(len(ocr_vqa_image))

vq_image = []
for data in vg_data:
    vq_image.append(data["image"])
print(len(vq_image))

gqa_image = []
for data in gqa_data:
    gqa_image.append(data["image"])
print(len(gqa_image))

all_images = textvqa_image + coco_image + ocr_vqa_image + vq_image + gqa_image
print(len(all_images))

2195
36410
8000
8642
7214
62461


In [11]:
mix_textvqa_data = []
for index, data in enumerate(textvqa_data[::50]):
    if index*50<1200:
        image_list = find_companions(data['image'], textvqa_image, 1)
    elif index*50<1550:
        image_list = find_companions(data['image'], textvqa_image, 2)
    elif index*50<1900:
        image_list = find_companions(data['image'], textvqa_image, 3)
    elif index*50<2195:
        image_list = find_companions(data['image'], textvqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index = len(image_list)+1
    image_length = len(image_list)
    data['image'] = image_list
    if image_length == 2:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 3:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 4:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    if image_length == 5:
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\nProvide a one-sentence caption for the provided image.\n', 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, provide a one-sentence caption for the provided image.\n')
    data["conversations"][1]['value'] = f'You only provided {image_length} images, Image{image_index} does not exist.'
    data["conversations"] = data["conversations"][:2]
    mix_textvqa_data.append(data)

print(len(mix_textvqa_data))

44


In [12]:
mix_coco_data = []
for index, data in enumerate(coco_data[::50]):
    if index*50<20000:
        image_list = find_companions(data['image'], coco_image, 1)
    elif index*50<26000:
        image_list = find_companions(data['image'], coco_image, 2)
    elif index*50<32000:
        image_list = find_companions(data['image'], coco_image, 3)
    elif index*50<36410:
        image_list = find_companions(data['image'], coco_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index = len(image_list)+1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data["conversations"][1]['value'] = f'You only provided {image_length} images, Image{image_index} does not exist.'
    data["conversations"] = data["conversations"][:2]
    mix_coco_data.append(data)
print(len(mix_coco_data))

729


In [13]:
mix_ocr_vqa_data = []
for index, data in enumerate(ocr_vqa_data[::50]):
    if index*50<4500:
        image_list = find_companions(data['image'], ocr_vqa_image, 1)
    elif index*50<5700:
        image_list = find_companions(data['image'], ocr_vqa_image, 2)
    elif index*50<6900:
        image_list = find_companions(data['image'], ocr_vqa_image, 3)
    elif index*50<8000:
        image_list = find_companions(data['image'], ocr_vqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index = len(image_list)+1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data["conversations"][1]['value'] = f'You only provided {image_length} images, Image{image_index} does not exist.'
    data["conversations"] = data["conversations"][:2]
    mix_ocr_vqa_data.append(data)
print(len(mix_ocr_vqa_data))

160


In [14]:
mix_vg_data = []
for index, data in enumerate(vg_data[::50]):
    if index*50<4800:
        image_list = find_companions(data['image'], vq_image, 1)
    elif index*50<6100:
        image_list = find_companions(data['image'], vq_image, 2)
    elif index*50<7400:
        image_list = find_companions(data['image'], vq_image, 3)
    elif index*50<8642:
        image_list = find_companions(data['image'], vq_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index = len(image_list)+1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data["conversations"][1]['value'] = f'You only provided {image_length} images, Image{image_index} does not exist.'
    data["conversations"] = data["conversations"][:2]
    mix_vg_data.append(data)
print(len(mix_vg_data))

173


In [15]:
mix_gqa_data = []
for index, data in enumerate(gqa_data[::50]):
    if index*50<3800:
        image_list = find_companions(data['image'], gqa_image, 1)
    elif index*50<5000:
        image_list = find_companions(data['image'], gqa_image, 2)
    elif index*50<6100:
        image_list = find_companions(data['image'], gqa_image, 3)
    elif index*50<7214:
        image_list = find_companions(data['image'], gqa_image, 4)
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])

    image_index = len(image_list)+1
    image_length = len(image_list)
    data['image'] = image_list
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                if image_length == 2:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 3:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 4:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                if image_length == 5:
                    convs['value'] = 'Image1:<image>\nImage2:<image>\nImage3:<image>\nImage4:<image>\nImage5:<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data["conversations"][1]['value'] = f'You only provided {image_length} images, Image{image_index} does not exist.'
    data["conversations"] = data["conversations"][:2]
    mix_gqa_data.append(data)
print(len(mix_gqa_data))

145


In [16]:
mix_data = mix_textvqa_data + mix_coco_data + mix_ocr_vqa_data + mix_vg_data + mix_gqa_data
print(len(mix_data))

1251


In [17]:
file_path = "./mix_llava62k_v3_reject_answer_1251.json"
with open(file_path, 'w') as file:
    json.dump(mix_data, file, indent=4)
print("Done")

Done


In [ ]:
"""
从 llava665k 中选取了 187k 的数据，并组合 多轮对话 拼图数据
"""

In [4]:
import random

def find_companions(input_str, str_list, count=1):
    if isinstance(input_str, list):
        input_set = set(input_str)
    else:
        input_set = {input_str}
    
    filtered_set = set(str_list) - input_set
    filtered_list = list(filtered_set)
    
    count = min(count, len(filtered_list))
    
    return random.sample(filtered_list, count)

# 示例
str_list = ["apple", "banana", "orange", "grape", "melon"]
input_str = ["apple", "grape"]
result = find_companions(input_str, str_list, count=2)
print(result)

['banana', 'orange']


In [5]:
from PIL import Image, ImageDraw, ImageFont

def resize_and_pad(image, target_size):
    """Resize and pad an image to the target size."""
    img_ratio = image.width / image.height
    target_ratio = target_size[0] / target_size[1]

    if img_ratio > target_ratio:
        new_width = target_size[0]
        new_height = int(new_width / img_ratio)
    else:
        new_height = target_size[1]
        new_width = int(new_height * img_ratio)

    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    new_image = Image.new("RGB", target_size, (255, 255, 255))
    # paste_x = (target_size[0] - new_width) // 2
    # paste_y = (target_size[1] - new_height) // 2
    paste_x = 0
    paste_y = 0
    new_image.paste(resized_image, (paste_x, paste_y))
    return new_image

def combine_images_with_labels(image_paths, output_path, target_size=(448, 448), margin=40):
    # # 获取提问的图像的大小
    # resize_size = Image.open(image_paths[image_index-1]).size
    # print(resize_size)
    # # Load images and resize them
    # images = [Image.open(image_path).resize(resize_size) for image_path in image_paths]
    # num_images = len(images)

    images = [resize_and_pad(Image.open(image_path), target_size) for image_path in image_paths]
    num_images = len(images)

    resize_size = target_size
    # Calculate combined image size
    if num_images == 2 or num_images == 3:
        combined_width = resize_size[0] * num_images + margin * (num_images - 1)
        combined_height = resize_size[1] + margin  # Add margin for labels
    elif num_images == 4:
        combined_width = resize_size[0] * 2 + margin
        combined_height = resize_size[1] * 2 + margin * 2  # Add margin for labels
    elif num_images == 6:
        combined_width = resize_size[0] * 3 + margin * 2
        combined_height = resize_size[1] * 2 + margin * 2  # Add margin for labels
    elif num_images == 9:
        combined_width = resize_size[0] * 3 + margin * 2
        combined_height = resize_size[1] * 3 + margin * 3  # Add margin for labels
    else:
        raise ValueError("The number of images must be 2, 3, 4, 6, or 9.")
    
    # Create new image with white background
    combined_image = Image.new('RGB', (combined_width, combined_height), (255, 255, 255))
    
    # Define positions for each image and label
    positions = []
    labels = []
    if num_images == 2 or num_images == 3:
        for i in range(num_images):
            x = i * (resize_size[0] + margin)
            y = 0+margin
            positions.append((x, y))
            labels.append(f"Image{i+1}")
    elif num_images == 4:
        for i in range(2):
            for j in range(2):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{2*i+j+1}")
    elif num_images == 6:
        for i in range(2):
            for j in range(3):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{3*i+j+1}")
    elif num_images == 9:
        for i in range(3):
            for j in range(3):
                x = j * (resize_size[0] + margin)
                y = i * (resize_size[1] + margin)+margin
                positions.append((x, y))
                labels.append(f"Image{3*i+j+1}")
    
    # Paste images and draw labels
    draw = ImageDraw.Draw(combined_image)
    font = ImageFont.truetype("Arial.ttf", 36)  # Load a larger TrueType font, or use another available font

    for img, pos, label in zip(images, positions, labels):
        combined_image.paste(img, pos)
        # Calculate label position in the left top corner
        label_x = pos[0] + 20  # Add some padding from the left edge
        label_y = pos[1] - 40  # Add some padding from the top edge
        draw.text((label_x, label_y), label, (0, 0, 0), font=font)
    
    # Save the combined image
    combined_image.save(output_path)


# # Example usage
# image_list = [
#     "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/car1.jpg",
#     "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/car2.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/newcar3.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/newcar4.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/bear.png",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
#     # "/mnt/petrelfs/liuziyu/RLHF/Observation/pics/apple.jpg",
# ]
# output_path = "./combined_image_9.png"
# combine_images_with_labels(image_list, output_path)
# print("Done")

In [6]:
import re
def contains_bbox(s):
    bbox_pattern = r'\[\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?\s*\]'
    match = re.search(bbox_pattern, s)
    if match:
        return True
    else:
        return False
contains_bbox("There is a bbox [0.92,0.32,0.12,0.24]")

True

In [7]:
all_images = []
for data in dpo_data:
    all_images.append(data["image"])
print(len(all_images))

187383


In [8]:
count_vg = 0
count_coco_bbox = 0
dpo_data_no_bbox = []
for item in dpo_data:
    if item["image"].split("/")[0]=="vg":
        count_vg += 1
    elif item["image"].split("/")[0]=="coco":
        flage_bbox = False
        for conv in item["conversations"]:
            conv_value = conv["value"]
            if contains_bbox(conv_value):
                flage_bbox = True
                break
        if flage_bbox == True:
            count_coco_bbox += 1
        else:
            dpo_data_no_bbox.append(item)
    else:
        dpo_data_no_bbox.append(item)
print(count_vg)
print(count_coco_bbox)
print(len(dpo_data_no_bbox))

25925
14535
146923


In [9]:
original_list = dpo_data_no_bbox
print(original_list[0]["id"])
random.shuffle(original_list)
print(original_list[0]["id"])
sublist1 = original_list[:48000]
sublist2 = original_list[48000:76000]
sublist3 = original_list[76000:104000]
sublist4 = original_list[104000:125000]
sublist5 = original_list[125000:146000]

print(len(sublist1), len(sublist2), len(sublist3), len(sublist4), len(sublist5))

000000319154
000000169014
48000 28000 28000 21000 21000


In [10]:
new_sublist2 = []
for i in range(0, len(sublist2), 2):
    pair = [sublist2[i], sublist2[i + 1]]
    new_sublist2.append(pair)
print(len(new_sublist2))

new_sublist3 = []
for i in range(0, len(sublist3), 2):
    pair = [sublist3[i], sublist3[i + 1]]
    new_sublist3.append(pair)
print(len(new_sublist3))

new_sublist4 = []
for i in range(0, len(sublist4), 3):
    pair = [sublist4[i], sublist4[i + 1], sublist4[i + 2]]
    new_sublist4.append(pair)
print(len(new_sublist4))

new_sublist5 = []
for i in range(0, len(sublist5), 3):
    pair = [sublist5[i], sublist5[i + 1], sublist5[i + 2]]
    new_sublist5.append(pair)
print(len(new_sublist5))

14000
14000
7000
7000


In [11]:
output_json_path = "./sublist1.json"
with open(output_json_path, 'w') as json_file:
    json.dump(sublist1, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

output_json_path = "./new_sublist2.json"
with open(output_json_path, 'w') as json_file:
    json.dump(new_sublist2, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

output_json_path = "./new_sublist3.json"
with open(output_json_path, 'w') as json_file:
    json.dump(new_sublist3, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

output_json_path = "./new_sublist4.json"
with open(output_json_path, 'w') as json_file:
    json.dump(new_sublist4, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

output_json_path = "./new_sublist5.json"
with open(output_json_path, 'w') as json_file:
    json.dump(new_sublist5, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

Data has been written to ./sublist1.json
Data has been written to ./new_sublist2.json
Data has been written to ./new_sublist3.json
Data has been written to ./new_sublist4.json
Data has been written to ./new_sublist5.json


In [12]:
with open("./sublist1.json", 'r', encoding='utf-8') as file:
    sublist1 = json.load(file)
print(len(sublist1))

ping_sublist1 = []
for index, data in tqdm(enumerate(sublist1)):
    
    image_list = find_companions(data['image'], all_images, 1)
    
    image_list.append(data['image'])
    random.shuffle(image_list)
    image_index = image_list.index(data['image'])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_multirounds/sublist1_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index += 1
    image_length = len(image_list)
    

    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                convs['value'] = '<image>\n' + f'In Image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                convs['value'] = f'In image{image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
    data['image'] = output_path
    ping_sublist1.append(data)

print(len(ping_sublist1))
output_json_path = "./ping_sublist1.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_sublist1, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

48000


48000it [39:41, 20.16it/s]


48000
Data has been written to ./ping_sublist1.json


In [13]:
with open("./new_sublist2.json", 'r', encoding='utf-8') as file:
    new_sublist2 = json.load(file)
print(len(new_sublist2))

ping_sublist2 = []
for index, data in tqdm(enumerate(new_sublist2)):
    data_image_list = [item["image"] for item in data]
    image_list = find_companions(data_image_list, all_images, 1)
    
    image_list.extend(data_image_list)
    random.shuffle(image_list)
    image_index_1 = image_list.index(data_image_list[0])
    image_index_2 = image_list.index(data_image_list[1])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_multirounds/sublist2_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index = [image_index_1+1, image_index_2+1]
    image_length = len(image_list)

    new_data = {}
    new_data["id"] = str(data[0]["id"]) + "_" + str(data[1]["id"])
    new_data['image'] = output_path
    new_data['conversations'] = []
    for i in range(len(data_image_list)):
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>\n', '')
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>', '')
        local_image_index = image_index[i]
        if i == 0:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = '<image>\n' + f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])
        else:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])

    ping_sublist2.append(new_data)

print(len(ping_sublist2))
output_json_path = "./ping_sublist2.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_sublist2, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

14000


14000it [14:31, 16.06it/s]


14000
Data has been written to ./ping_sublist2.json


In [14]:
with open("./new_sublist3.json", 'r', encoding='utf-8') as file:
    new_sublist3 = json.load(file)
print(len(new_sublist3))

ping_sublist3 = []
for index, data in tqdm(enumerate(new_sublist3)):
    data_image_list = [item["image"] for item in data]
    image_list = find_companions(data_image_list, all_images, 2)
    
    image_list.extend(data_image_list)
    random.shuffle(image_list)
    image_index_1 = image_list.index(data_image_list[0])
    image_index_2 = image_list.index(data_image_list[1])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_multirounds/sublist3_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index = [image_index_1+1, image_index_2+1]
    image_length = len(image_list)

    new_data = {}
    new_data["id"] = str(data[0]["id"]) + "_" + str(data[1]["id"])
    new_data['image'] = output_path
    new_data['conversations'] = []
    for i in range(len(data_image_list)):
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>\n', '')
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>', '')
        local_image_index = image_index[i]
        if i == 0:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = '<image>\n' + f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])
        else:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])

    ping_sublist3.append(new_data)

print(len(ping_sublist3))
output_json_path = "./ping_sublist3.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_sublist3, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

14000


14000it [17:23, 13.42it/s]


14000
Data has been written to ./ping_sublist3.json


In [15]:
with open("./new_sublist4.json", 'r', encoding='utf-8') as file:
    new_sublist4 = json.load(file)
print(len(new_sublist4))

ping_sublist4 = []
for index, data in tqdm(enumerate(new_sublist4)):
    data_image_list = [item["image"] for item in data]
    image_list = find_companions(data_image_list, all_images, 3)
    
    image_list.extend(data_image_list)
    random.shuffle(image_list)
    image_index_1 = image_list.index(data_image_list[0])
    image_index_2 = image_list.index(data_image_list[1])
    image_index_3 = image_list.index(data_image_list[2])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_multirounds/sublist4_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index = [image_index_1+1, image_index_2+1, image_index_3+1]
    image_length = len(image_list)

    new_data = {}
    new_data["id"] = str(data[0]["id"]) + "_" + str(data[1]["id"]) + "_" + str(data[2]["id"])
    new_data['image'] = output_path
    new_data['conversations'] = []
    for i in range(len(data_image_list)):
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>\n', '')
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>', '')
        local_image_index = image_index[i]
        if i == 0:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = '<image>\n' + f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])
        else:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])

    ping_sublist4.append(new_data)

print(len(ping_sublist4))
output_json_path = "./ping_sublist4.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_sublist4, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

7000


7000it [11:52,  9.82it/s]


7000
Data has been written to ./ping_sublist4.json


In [16]:
with open("./new_sublist5.json", 'r', encoding='utf-8') as file:
    new_sublist5 = json.load(file)
print(len(new_sublist5))

ping_sublist5 = []
for index, data in tqdm(enumerate(new_sublist5)):
    data_image_list = [item["image"] for item in data]
    image_list = find_companions(data_image_list, all_images, 6)
    
    image_list.extend(data_image_list)
    random.shuffle(image_list)
    image_index_1 = image_list.index(data_image_list[0])
    image_index_2 = image_list.index(data_image_list[1])
    image_index_3 = image_list.index(data_image_list[2])
    new_image_list = []
    for img_path in image_list:
        new_image_list.append('/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + img_path)
    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_combined_pics_multirounds/sublist5_{index}.jpg'
    combine_images_with_labels(new_image_list, output_path)

    image_index = [image_index_1+1, image_index_2+1, image_index_3+1]
    image_length = len(image_list)

    new_data = {}
    new_data["id"] = str(data[0]["id"]) + "_" + str(data[1]["id"]) + "_" + str(data[2]["id"])
    new_data['image'] = output_path
    new_data['conversations'] = []
    for i in range(len(data_image_list)):
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>\n', '')
        data[i]["conversations"][0]['value'] = data[i]["conversations"][0]['value'].replace('<image>', '')
        local_image_index = image_index[i]
        if i == 0:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = '<image>\n' + f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])
        else:
            for convs_index, convs in enumerate(data[i]["conversations"]):
                if convs['from'] == 'human':
                    if convs_index == 0:
                        convs['value'] = f'In Image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
                    else:
                        convs['value'] = f'In image{local_image_index}, ' + convs['value'][0].lower() + convs['value'][1:]
            new_data['conversations'].extend(data[i]["conversations"])

    ping_sublist5.append(new_data)

print(len(ping_sublist5))
output_json_path = "./ping_sublist5.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_sublist5, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

7000


7000it [16:21,  7.13it/s]


7000
Data has been written to ./ping_sublist5.json


In [17]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/ping_sublist1.json", 'r', encoding='utf-8') as file:
    ping_sublist1 = json.load(file)
print(len(ping_sublist1))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/ping_sublist2.json", 'r', encoding='utf-8') as file:
    ping_sublist2 = json.load(file)
print(len(ping_sublist2))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/ping_sublist3.json", 'r', encoding='utf-8') as file:
    ping_sublist3 = json.load(file)
print(len(ping_sublist3))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/ping_sublist4.json", 'r', encoding='utf-8') as file:
    ping_sublist4 = json.load(file)
print(len(ping_sublist4))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/ping_sublist5.json", 'r', encoding='utf-8') as file:
    ping_sublist5 = json.load(file)
print(len(ping_sublist5))

ping_llava187k_90k = ping_sublist1 + ping_sublist2 + ping_sublist3 + ping_sublist4 + ping_sublist5
print(len(ping_llava187k_90k))
output_json_path = "./ping_llava187k_90k.json"
with open(output_json_path, 'w') as json_file:
    json.dump(ping_llava187k_90k, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

48000
14000
14000
7000
7000
90000
Data has been written to ./ping_llava187k_90k.json
